In [327]:
import numpy as np
import pandas as pd
lotto = pd.read_csv('/Users/jaeyoungcho/Desktop/lotto.csv',sep = ',',encoding='CP949')

In [328]:
# 데이터 전처리 및 추가
def lotto_data(lotto):
    
    lotto.drop(['Unnamed: 0'],axis = 1, inplace = True)
    lotto.columns = ['날짜','회차','당첨번호','보너스번호']
    
    dict_date = {}
    for i in lotto['날짜']:
        x = []
        for j in i:
            if ord(j) <= ord('9') and ord(j) >= ord('0'):
                x.append(j)
        temp = ''.join(x)
        dict_date[i] = temp

    lotto['날짜'].replace(dict_date, inplace = True)
    
    x = []
    for j in lotto['당첨번호']:
        j = j.replace('\n',',').strip(',')
        x.append(j)
        
    a,b,c,d,e,f = [],[],[],[],[],[]
    
    for i in x:
        i = i.split(sep= ',')
        a.append(i[0])
        b.append(i[1])
        c.append(i[2])
        d.append(i[3])
        e.append(i[4])
        f.append(i[5])
    
    lotto['1'] = a
    lotto['2'] = b
    lotto['3'] = c
    lotto['4'] = d
    lotto['5'] = e
    lotto['6'] = f
    lotto.drop(['당첨번호'],axis = 1, inplace = True)
    
    summ = []
    multi = []
    for i in range(0,936):
        summ.append(lotto[lotto.columns[3:]].loc[i].astype(np.int32).sum())
        z = 1
        for j in lotto[lotto.columns[3:]].loc[i].astype(np.int32):
            z *= j
        multi.append(z)
    
#     lotto['합'] = pd.Series(summ)
#     lotto['곱'] = pd.Series(np.array(multi).flatten())
    
#     lotto = lotto[['날짜','회차','보너스번호','합','곱','첫 번째','두 번째','세 번째','넷 번째','다섯 번째','여섯 번째']]
    lotto = lotto[['날짜','회차','보너스번호','1','2','3','4','5','6']]
    
    return lotto

In [329]:
def lotto_Standard(lotto):
    from sklearn.preprocessing import StandardScaler
    SD = StandardScaler()
    lotto = pd.DataFrame(SD.fit_transform(lotto),columns=lotto.columns)
    
    return lotto

In [330]:
lotto = lotto_data(lotto)
lotto = lotto_Standard(lotto)

In [331]:
lotto

,날짜,회차,보너스번호,1,2,3,4,5,6
0,-1.803725,-1.730201,-0.479642,0.629850,1.440219,1.208883,0.881644,0.603005,0.092475
1,-1.803590,-1.726500,-1.539547,0.440956,-0.010852,0.140030,-0.163598,-0.122461,0.468806
2,-1.803455,-1.722799,0.580262,0.818745,0.424469,-0.127183,-0.686219,-0.847927,-1.601017
3,-1.803320,-1.719098,-1.539547,1.385429,2.020647,1.342490,0.620333,1.038285,0.468806
4,-1.631892,-1.715397,-1.463839,1.763218,1.585326,1.208883,1.796230,1.183378,0.468806
...,...,...,...,...,...,...,...,...,...
931,1.668915,1.715397,-1.312424,-1.070201,-1.026601,-0.661610,1.273609,0.603005,-0.283857
932,1.669050,1.719098,1.110214,3.085480,2.020647,1.208883,0.620333,0.457912,1.033303
933,1.669186,1.722799,-0.782472,-1.070201,-1.461923,1.342490,0.881644,0.457912,-0.095691
934,1.669321,1.726500,-0.328227,-0.503517,-0.446173,0.006423,0.750988,0.748098,0.845138


In [373]:
for i in range(1,7):
    i = str(i)
    X,y = [lotto[['날짜','회차']],lotto[i]]
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)
    
    from sklearn.model_selection import GridSearchCV
    params = [
        {'kernel':['rbf','poly'],
         'gamma' : [0.1,0.5,1.0],
         'C':[0.1,1,3,5,7,10]
    }]

    from sklearn.svm import SVR
    sv = SVR()
    grid = GridSearchCV(sv,params,cv=5,scoring='neg_mean_squared_error',return_train_score=True)
    grid.fit(X_train,y_train)
    a = grid.best_params_['C']
    b = grid.best_params_['gamma']
    c = grid.best_params_['kernel']
    
    sv = SVR(C = a,gamma = b, kernel = c)
    sv.fit(X_train,y_train)
    
    print('{}번째 수의 훈련용 데이터 점수 : {}'.format(i,sv.score(X_train,y_train)))
    print('{}번째 수의 시험용 데이터 점수 : {}'.format(i,sv.score(X_test,y_test)))
    print('{}번째 수의 937회차 예측 값 : {}'.format(i,sv.predict()))
    print()

1번째 수의 훈련용 데이터 점수 : -0.07722165591839314
1번째 수의 시험용 데이터 점수 : -0.10573519585562563
2번째 수의 훈련용 데이터 점수 : -0.01734030697267075
2번째 수의 시험용 데이터 점수 : -0.00026349862936414503
3번째 수의 훈련용 데이터 점수 : -0.0015924641438378462
3번째 수의 시험용 데이터 점수 : -0.005286739770595572
4번째 수의 훈련용 데이터 점수 : -0.0026356251906785033
4번째 수의 시험용 데이터 점수 : -0.01408182811515224
5번째 수의 훈련용 데이터 점수 : -0.02414204595752567
5번째 수의 시험용 데이터 점수 : -0.05580605997188992
6번째 수의 훈련용 데이터 점수 : -0.046056964081590435
6번째 수의 시험용 데이터 점수 : -0.014082166650620787


In [287]:
def MLP(lotto):
    modeling = []
    for i in range(6,12):
        X,y = [lotto[lotto.columns[:5]],lotto[lotto.columns[i]]]
        from sklearn.model_selection import train_test_split

        X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)
        
        import tensorflow as tf
        from tensorflow import keras

        model = tf.keras.Sequential([
            tf.keras.layers.Dense(30,activation='relu',input_shape = X_train.shape[1:]),
            tf.keras.layers.Dense(1)
        ])
        
        model.compile(loss = 'mean_squared_error',optimizer = 'sgd')

In [288]:
model = MLP(lotto)

Epoch 1/30
22/22 [==============================] - 0s 446us/step - loss: 0.7908
Epoch 2/30
22/22 [==============================] - 0s 434us/step - loss: 0.4569
Epoch 3/30
22/22 [==============================] - 0s 391us/step - loss: 0.3931
Epoch 4/30
22/22 [==============================] - 0s 419us/step - loss: 0.3748
Epoch 5/30
22/22 [==============================] - 0s 412us/step - loss: 0.3646
Epoch 6/30
22/22 [==============================] - 0s 397us/step - loss: 0.3558
Epoch 7/30
22/22 [==============================] - 0s 423us/step - loss: 0.3497
Epoch 8/30
22/22 [==============================] - 0s 444us/step - loss: 0.3442
Epoch 9/30
22/22 [==============================] - 0s 462us/step - loss: 0.3416
Epoch 10/30
22/22 [==============================] - 0s 470us/step - loss: 0.3370
Epoch 11/30
22/22 [==============================] - 0s 515us/step - loss: 0.3342
Epoch 12/30
22/22 [==============================] - 0s 503us/step - loss: 0.3316
Epoch 13/30
22/22 [======